## Imports

In [ ]:
import os
import sys
sys.path.insert(0, "/Users/neeraja/fiftyone")
os.environ["PYTHONPATH"] = "/Users/neeraja/fiftyone:/Users/neeraja/fiftyone-brain"

In [ ]:
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [ ]:
import fiftyone as fo
import fiftyone.operators as foo
import fiftyone.brain as fob
import fiftyone.zoo as foz

In [ ]:
sys.path.append(os.path.dirname(os.getcwd()))
from annoprop import propagate_annotations

In [ ]:
dataset = fo.load_dataset("basketball_frames")
dataset_slice = dataset.load_saved_view("spinning_part1")
# dataset_slice = dataset.load_saved_view("side_top_layup")
# dataset_slice = dataset.load_saved_view("underbasket_reverse_layup")
print(len(dataset_slice))

## [temp] Demo:

In [ ]:
import fiftyone as fo
import fiftyone.operators as foo
import fiftyone.brain as fob
import fiftyone.zoo as foz
from fiftyone import ViewField as F

In [ ]:
dataset = fo.load_dataset("basketball_frames")

if "exemplar_test_allhands" in dataset.get_field_schema():
    dataset.delete_sample_field("exemplar_test_allhands")
if "human_labels" in dataset.get_field_schema():
    dataset.delete_sample_field("human_labels")

# annotated_view = dataset.match(F("ha_test_1") != None)
# demo_view = annotated_view.take(100, seed=5)

demo_view = dataset.load_saved_view("spinning_part1")

session = fo.launch_app(demo_view)

In [ ]:
# Step 1: Run the exemplar extraction operator
# UI

# or mock it here
ctx1 = {
    "dataset": dataset,
    "view": demo_view,
    "params": {
        "method": "zcore:embeddings_resnet18",
        "exemplar_frame_field": "exemplar_test_allhands",
        "max_fraction_exemplars": 0.27,
    },
}
exemplar_result = foo.execute_operator(
    "@neerajaabhyankar/video-exemplar-frames-plugin/extract_exemplar_frames",
    ctx1
)

In [ ]:
# Step 2: For the chosen frames, copy the ha_test_1 annotation
for sample in demo_view:
    if sample["exemplar_test_allhands"]["is_exemplar"]:
        sample["human_labels"] = sample["ha_test_1"]
        sample.save()

In [ ]:
# Step 3: Run the propagation operator
# UI

# or mock it here
ctx2 = {
    "dataset": dataset,
    "view": demo_view,
    "params": {
        "exemplar_frame_field": "exemplar_test_allhands",
        "input_annotation_field": "human_labels",
        "output_annotation_field": "human_labels_propagated",
    },
}
anno_prop_result = foo.execute_operator(
    "@neerajaabhyankar/video-exemplar-frames-plugin/propagate_annotations_from_exemplars",
    ctx2
)

## Run Annotation Propagation Methods

For now: pick every alternate frame as an exemplar

Later: "Cross-Propagation"
Choose each frame as an exemplar turn-by-turn
And record the accuracy of the propagation from it

In [ ]:
# Set the exemplar frame field and assignments
dataset.add_sample_field("exemplar_test", fo.DictField)
exemplar_id = dataset.first().id
for ii, sample in enumerate(dataset_slice.sort_by("frame_number")):
    if ii % 2 == 0:
        sample["exemplar_test"] = {
            "is_exemplar": True,
            "exemplar_assignment": []
        }
        exemplar_id = sample.id
    else:
        sample["exemplar_test"] = {
            "is_exemplar": False,
            "exemplar_assignment": [exemplar_id]
        }
    sample.save()

In [ ]:
score = propagate_annotations(
    dataset_slice,
	exemplar_frame_field="exemplar_test", 
	input_annotation_field="ha_test_1",
	output_annotation_field="ha_test_1_propagated",
)
print(f"Score: {score}")

In [ ]:
session = fo.launch_app(dataset_slice)
session.wait()

## Inspect Embeddings

In [ ]:
from annoprop_algos import setup_siamfc

tracker = setup_siamfc()
_ = tracker.net.eval()

In [ ]:
all_embeddings = []

for sample in dataset_slice:
    img = cv2.imread(sample.filepath)
    # Convert to torch tensor and process through backbone (similar to siamfc.py:159-166)
    # but without cropping - send entire image
    x = torch.from_numpy(img).to(
        tracker.device).permute(2, 0, 1).unsqueeze(0).float()
    embedding = tracker.net.backbone(x)
    embedding = embedding.detach().cpu().numpy().squeeze(0)
    # embedding is of shape (256, 35, 70)
    # with the latter two being spatial dimensions
    all_embeddings.append(embedding)

all_embeddings = np.array(all_embeddings)  # Shape: (45, 256, 35, 70)
                                           # where 45 is the number of samples

### Center of the frame Only

In [ ]:
center_embedding = all_embeddings[:, :, 17, 35]

tsne = TSNE(n_components=2, init='pca', random_state=501)
tsne_embedding = tsne.fit_transform(center_embedding)
plt.scatter(tsne_embedding[:, 0], tsne_embedding[:, 1])
plt.show()

In [ ]:
# import umap
# umap_model = umap.UMAP(n_neighbors=3, min_dist=0.1, n_components=2)
# umap_embedding = umap_model.fit_transform(center_embedding)
# plt.scatter(umap_embedding[:, 0], umap_embedding[:, 1])
# plt.show()

### A 10x10 block around the center

In [ ]:
import colorstamps

In [ ]:
# Extract 10x10 block around center (17, 35)
block_embeddings = all_embeddings[:, :, 12:22, 30:40]  # Shape: (45, 256, 10, 10)

# Flatten to (4500, 256): (num_samples * 10 * 10, 256)
num_samples = block_embeddings.shape[0]
flattened_embeddings = block_embeddings.transpose(0, 2, 3, 1).reshape(-1, 256)  # Shape: (4500, 256)

In [ ]:
x_coords = np.linspace(-1, 1, 10)
y_coords = np.linspace(-1, 1, 10)
X, Y = np.meshgrid(x_coords, y_coords)
rgb, stamp = colorstamps.apply_stamp(
    X, Y, 'peak',
    vmin_0=-1, vmax_0=1,
    vmin_1=-1, vmax_1=1
)  # rgb has Shape: (10, 10, 3)
flattened_colormap = rgb.reshape(-1, 3)  # Shape: (100, 3)

In [ ]:
# Repeat the colormap for each sample to match the flattened embeddings
colors = np.tile(flattened_colormap, (num_samples, 1))  # Shape: (4500, 3)

In [ ]:
# Apply t-SNE
tsne_block = TSNE(n_components=2, init='pca', random_state=501)
tsne_embedding_block = tsne_block.fit_transform(flattened_embeddings)

In [ ]:
# Plot with colors corresponding to 2D position in the 10x10 block
plt.figure(figsize=(4, 4))
plt.scatter(tsne_embedding_block[:, 0], tsne_embedding_block[:, 1], c=colors, s=5, alpha=0.6)
plt.title('t-SNE Visualization of 10x10 Block Embeddings (colored by spatial position)')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
# plot the rgb matrix as a legend on the side
plt.figure(figsize=(3, 3))
plt.imshow(rgb)
plt.title("Color Legend for the 10x10 Block")
plt.show()

### Deltas between neighboring embeddings

In [ ]:
plt.figure(figsize=(16, 8))

for ii in range(1, all_embeddings.shape[0]):
    plt.subplot(5, 9, ii)
    delta = all_embeddings[ii] - all_embeddings[ii-1]
    delta_norm = np.linalg.norm(delta, axis=0)
    plt.imshow(delta_norm)
    plt.title(f"Frame {ii} - {ii-1}")
    # plt.title(f"Frame {ii} - {ii-1}: {np.max(delta_norm)}")
    plt.xticks([])
    plt.yticks([])

plt.subplot(5, 9, 45)
im = plt.imshow(delta_norm)  # dummy image to attach the colorbar
plt.axis('off')
plt.colorbar(im, orientation='vertical', fraction=1.0)
plt.tight_layout()
plt.show()

In [ ]:
# plot the images themselves
plt.figure(figsize=(16, 8))
for ii, sample in enumerate(dataset_slice):
    plt.subplot(5, 9, ii+1)
    image = cv2.imread(sample.filepath)
    plt.imshow(image[:, :, ::-1])
    plt.title(f"Frame {ii}")
    plt.xticks([])
    plt.yticks([])
plt.tight_layout()
plt.show()

For every patch in frame ii, does there exist a patch in ii-1 that has a delta < a threshold?<br>
Can we do a fast search?

In [ ]:
PATCH_NBD = max(all_embeddings.shape[2], all_embeddings.shape[3]) // 10

In [ ]:
max_min_delta_from_previous = []
for kk in range(1, len(all_embeddings)):
    img1 = all_embeddings[kk-1]
    img2 = all_embeddings[kk]
    min_deltas = []
    for ii in range(img2.shape[1]):
        for jj in range(img2.shape[2]):
            patch2 = img2[:, ii, jj]
            img1_neighborhood = img1[:, max(0, ii-PATCH_NBD):min(img1.shape[1], ii+PATCH_NBD), max(0, jj-PATCH_NBD):min(img1.shape[2], jj+PATCH_NBD)]
            patch_deltas = np.linalg.norm(img1_neighborhood - patch2[:, np.newaxis, np.newaxis], axis=0)
            min_deltas.append(np.min(patch_deltas))
    # print(f"max(min(patch_deltas)) = {np.max(min_deltas)}")
    max_min_delta_from_previous.append(np.max(min_deltas))

In [ ]:
# plot the images themselves
plt.figure(figsize=(16, 8))
for ii, sample in enumerate(dataset_slice):
    if ii == 0:
        continue
    # plt.subplot(5, 9, ii)
    plt.subplot(3, 8, ii)
    image = cv2.imread(sample.filepath)
    plt.imshow(image[:, :, ::-1])
    plt.title(f"Frame {ii}; d = {max_min_delta_from_previous[ii-1]:.1f}")
    plt.xticks([])
    plt.yticks([])
plt.tight_layout()
plt.show()

What is a good "threshold"? Does it depend on:
1. The image's scale (I guess not, given we're taking the min over a fixed nbd)
2. ..?

### Hausdorff Distance between images

In [ ]:
NN, DD, HH, WW = all_embeddings.shape

In [ ]:
PATCH_NBD = max(HH, WW) // 10

In [ ]:
def hausdorff_distance_between_images_nbdbased(img1, img2):
    min_deltas = []
    for hh in range(HH):
        for ww in range(WW):
            patch2 = img2[:, hh, ww]
            img1_neighborhood = img1[:, max(0, hh-PATCH_NBD):min(img1.shape[1], hh+PATCH_NBD), max(0, ww-PATCH_NBD):min(img1.shape[2], ww+PATCH_NBD)]
            patch_deltas = np.linalg.norm(img1_neighborhood - patch2[:, np.newaxis, np.newaxis], axis=0)
            min_deltas.append(np.min(patch_deltas))
    return np.max(min_deltas)

In [ ]:
def hausdorff_distance_between_images(img1, img2):
    min_deltas = []
    for hh in range(HH):
        for ww in range(WW):
            patch2 = img2[:, hh, ww]
            patch_deltas = np.linalg.norm(img1 - patch2[:, np.newaxis, np.newaxis], axis=0)
            min_deltas.append(np.min(patch_deltas))
    return np.max(min_deltas)

In [ ]:
def hausdorff_distance_between_images_vectorized(img1, img2):
    img1_patches = img1.reshape(img1.shape[0], -1).T     # (n_patches, DD)
    img2_patches = img2.reshape(img2.shape[0], -1).T     # (n_patches, DD)
    dists = np.linalg.norm(
        img1_patches[None, :, :] - img2_patches[:, None, :], axis=-1
    )
    min_dists = np.min(dists, axis=1)
    return min_dists.max()


In [ ]:
# import time
# start_time = time.time()
# print(hausdorff_distance_between_images(all_embeddings[10], all_embeddings[1]))
# print(f"Time taken: {time.time() - start_time} seconds")
# start_time = time.time()
# print(hausdorff_distance_between_images_vectorized(all_embeddings[10], all_embeddings[1]))
# print(f"Time taken: {time.time() - start_time} seconds")
# start_time = time.time()
# print(hausdorff_distance_between_images_nbdbased(all_embeddings[10], all_embeddings[1]))
# print(f"Time taken: {time.time() - start_time} seconds")

In [ ]:
hausdorff_matrix = np.zeros((NN, NN))
for ii in range(NN):
    for jj in range(ii+1, NN):
        hausdorff_matrix[ii, jj] = hausdorff_distance_between_images_nbdbased(all_embeddings[ii], all_embeddings[jj])

# fill up the lower triangle
hausdorff_matrix = np.triu(hausdorff_matrix) + np.triu(hausdorff_matrix, 1).T

#### Visualize

In [ ]:
# tsne using precomputed hausdorff distances
tsne_hausdorff = TSNE(n_components=2, init="random", random_state=42, metric="precomputed", perplexity=10)
tsne_hausdorff_embedding = tsne_hausdorff.fit_transform(hausdorff_matrix)
plt.scatter(tsne_hausdorff_embedding[:, 0], tsne_hausdorff_embedding[:, 1], s=4)
for i, (x, y) in enumerate(zip(tsne_hausdorff_embedding[:, 0], tsne_hausdorff_embedding[:, 1])):
    plt.annotate(
        str(i), (x, y), xytext=(2, 2),
        textcoords='offset points', ha='left', va='bottom',
        fontsize=8,
    )
plt.show()

In [ ]:
# umap using precomputed hausdorff distances
import umap
umap_hausdorff = umap.UMAP(n_neighbors=3, min_dist=0.1, n_components=2, metric="precomputed")
umap_hausdorff_embedding = umap_hausdorff.fit_transform(hausdorff_matrix)
plt.scatter(umap_hausdorff_embedding[:, 0], umap_hausdorff_embedding[:, 1])
plt.show()

#### Cluster

In [ ]:
def euclidean_or_not(D, tol=1e-8):
    """
    Check whether a distance matrix is Euclidean.

    Args:
        D (np.ndarray): NxN symmetric distance matrix with zeros on diagonal
        tol (float): numerical tolerance for negative eigenvalues

    Returns:
        is_euclidean (bool)
        stats (dict): diagnostics you actually care about
    """
    D = np.asarray(D)
    assert D.ndim == 2 and D.shape[0] == D.shape[1], "D must be NxN"
    assert np.allclose(D, D.T, atol=tol), "D must be symmetric"
    assert np.allclose(np.diag(D), 0, atol=tol), "Diagonal must be zero"

    N = D.shape[0]

    # Double-centering
    J = np.eye(N) - np.ones((N, N)) / N
    B = -0.5 * J @ (D ** 2) @ J

    # Eigenvalues of Gram matrix
    eigvals = np.linalg.eigvalsh(B)

    neg_eigs = eigvals[eigvals < -tol]
    pos_eigs = eigvals[eigvals > tol]

    neg_energy = np.sum(np.abs(neg_eigs))
    total_energy = np.sum(np.abs(eigvals))

    stats = {
        "min_eigenvalue": eigvals.min(),
        "num_negative_eigenvalues": len(neg_eigs),
        "negative_energy_ratio": (
            neg_energy / total_energy if total_energy > 0 else 0.0
        ),
        "embedding_dimension": len(pos_eigs),
        "eigenvalues": eigvals,
    }

    is_euclidean = len(neg_eigs) == 0

    return is_euclidean, stats

In [ ]:
is_euclidean, stats = euclidean_or_not(hausdorff_matrix)
print(is_euclidean)
print(stats)
plt.imshow(hausdorff_matrix)
plt.colorbar()
plt.show()


In [ ]:
def project_to_nearest_euclidean(D, tol=1e-8, return_embedding=False):
    """
    Project a distance matrix to the nearest Euclidean distance matrix
    using classical MDS eigenvalue clipping.

    Args:
        D (np.ndarray): NxN symmetric distance matrix
        tol (float): eigenvalue threshold
        return_embedding (bool): if True, also return coordinates

    Returns:
        D_euc (np.ndarray): projected Euclidean distance matrix
        X (np.ndarray, optional): Nxk Euclidean embedding
    """
    D = np.asarray(D)
    N = D.shape[0]

    assert D.shape[0] == D.shape[1], "D must be NxN"
    assert np.allclose(D, D.T, atol=tol), "D must be symmetric"
    assert np.allclose(np.diag(D), 0, atol=tol), "Diagonal must be zero"

    # Double-centering
    J = np.eye(N) - np.ones((N, N)) / N
    B = -0.5 * J @ (D ** 2) @ J

    # Eigen-decomposition
    eigvals, eigvecs = np.linalg.eigh(B)

    # Clip negative eigenvalues
    eigvals_clipped = np.clip(eigvals, 0, None)

    # Reconstruct Gram matrix
    B_psd = eigvecs @ np.diag(eigvals_clipped) @ eigvecs.T

    # Recover coordinates
    pos = eigvals_clipped > tol
    X = eigvecs[:, pos] @ np.diag(np.sqrt(eigvals_clipped[pos]))

    # Reconstruct Euclidean distances
    sq_norms = np.sum(X ** 2, axis=1, keepdims=True)
    D_euc_sq = sq_norms + sq_norms.T - 2 * (X @ X.T)
    D_euc_sq = np.maximum(D_euc_sq, 0.0)
    D_euc = np.sqrt(D_euc_sq)

    if return_embedding:
        return D_euc, X
    else:
        return D_euc


In [ ]:
hausdorff_matrix_euclidean, mds_embedding = project_to_nearest_euclidean(hausdorff_matrix, return_embedding=True)
plt.imshow(hausdorff_matrix_euclidean)
plt.colorbar()
plt.show()


In [ ]:
euclidean_delta = hausdorff_matrix_euclidean - hausdorff_matrix
plt.imshow(euclidean_delta)
plt.colorbar()
plt.show()

In [ ]:
# # # np.unravel_index(np.argmax(hausdorff_matrix), hausdorff_matrix.shape)

# for ii, sample in enumerate(dataset_slice):
#     if ii == 13:
#         prev_image = cv2.imread(sample.filepath)
#     if ii == 14:
#         next_image = cv2.imread(sample.filepath)
#         break

# plt.imshow(prev_image[:, :, ::-1])
# plt.show()

# plt.imshow(next_image[:, :, ::-1])
# plt.show()

#### Embeddings derived from the Hausdorff Matrix

In [ ]:
MDS_DIM = 8         # target embedding dimension
MDS_EIG_TOL = 1e-8  # eigenvalue threshold

In [ ]:
def compute_classical_mds_embedding(D, dim=MDS_DIM, eig_tol=MDS_EIG_TOL):
    """
    Classical MDS embedding from a distance matrix.

    Args:
        D (np.ndarray): NxN symmetric distance matrix
        dim (int): target embedding dimension
        eig_tol (float): eigenvalue threshold

    Returns:
        X (np.ndarray): Nxk embedding (k <= dim)
        eigvals (np.ndarray): eigenvalues used
    """
    D = np.asarray(D)
    N = D.shape[0]

    J = np.eye(N) - np.ones((N, N)) / N
    B = -0.5 * J @ (D ** 2) @ J

    eigvals, eigvecs = np.linalg.eigh(B)

    # Sort descending
    idx = np.argsort(eigvals)[::-1]
    eigvals = eigvals[idx]
    eigvecs = eigvecs[:, idx]

    # Keep strictly positive eigenvalues
    pos = eigvals > eig_tol
    eigvals = eigvals[pos]
    eigvecs = eigvecs[:, pos]

    # Truncate to target dim
    k = min(dim, eigvals.shape[0])
    eigvals = eigvals[:k]
    eigvecs = eigvecs[:, :k]

    X = eigvecs @ np.diag(np.sqrt(eigvals))
    return X, eigvals


In [ ]:
SPEC_DIM = 16
SPEC_SIGMA = None     # if None, median heuristic
SPEC_NORMALIZE = True

In [ ]:
def compute_spectral_embedding(D, dim=SPEC_DIM, sigma=SPEC_SIGMA, normalize=SPEC_NORMALIZE):
    """
    Spectral embedding from a distance matrix via RBF kernel.

    Args:
        D (np.ndarray): NxN symmetric distance matrix
        dim (int): embedding dimension
        sigma (float or None): kernel bandwidth
        normalize (bool): use normalized Laplacian

    Returns:
        X (np.ndarray): Nx(dim) spectral embedding
        eigvals (np.ndarray): eigenvalues
    """
    D = np.asarray(D)
    N = D.shape[0]

    # Bandwidth selection
    if sigma is None:
        sigma = np.median(D[D > 0])

    # Similarity matrix
    K = np.exp(-(D ** 2) / (2 * sigma ** 2))
    np.fill_diagonal(K, 0.0)

    # Degree matrix
    d = np.sum(K, axis=1)
    D_inv_sqrt = np.diag(1.0 / np.sqrt(d + 1e-12))

    if normalize:
        L = np.eye(N) - D_inv_sqrt @ K @ D_inv_sqrt
    else:
        L = np.diag(d) - K

    # Eigen-decomposition (smallest eigenvalues)
    eigvals, eigvecs = np.linalg.eigh(L)

    # Skip trivial eigenvector
    X = eigvecs[:, 1:dim+1]
    eigvals = eigvals[1:dim+1]

    return X, eigvals


In [ ]:
mds_embedding, _ = compute_classical_mds_embedding(hausdorff_matrix)
spectral_embedding, _ = compute_spectral_embedding(hausdorff_matrix)

In [ ]:
mds_embedding.shape, spectral_embedding.shape

In [ ]:
# visualize the embeddings with t-SNE
tsne_mds = TSNE(n_components=2, init='pca', random_state=42, perplexity=10)
tsne_mds_embedding = tsne_mds.fit_transform(mds_embedding)
plt.scatter(tsne_mds_embedding[:, 0], tsne_mds_embedding[:, 1], s=4)
for i, (x, y) in enumerate(zip(tsne_mds_embedding[:, 0], tsne_mds_embedding[:, 1])):
    plt.annotate(
        str(i), (x, y), xytext=(2, 2),
        textcoords='offset points', ha='left', va='bottom',
        fontsize=8,
    )
plt.show()

tsne_spectral = TSNE(n_components=2, init='pca', random_state=42, perplexity=10)
tsne_spectral_embedding = tsne_spectral.fit_transform(spectral_embedding)
plt.scatter(tsne_spectral_embedding[:, 0], tsne_spectral_embedding[:, 1], s=4)
for i, (x, y) in enumerate(zip(tsne_spectral_embedding[:, 0], tsne_spectral_embedding[:, 1])):
    plt.annotate(
        str(i), (x, y), xytext=(2, 2),
        textcoords='offset points', ha='left', va='bottom',
        fontsize=8,
    )
plt.show()

The `spinning` view should ideally lie on a 1d manifold.<br>
The t-SNE of the MDS embedding kinda does that (actually seems very similar to the t-SNE of the precomputed distances themselves -- maybe t-SNE does MDS under the hood).<br>
But the spectral embedding looks pretty random -- I guess not a good fit.

In [ ]:
mds_embedding.shape

In [ ]:
# # add the MDS embedding to the dataset
# embedding_field_name = "embeddings_hausdorff_nbd_mds_8"
# dataset_slice._dataset.add_sample_field(embedding_field_name, fo.VectorField, shape=(MDS_DIM,))

# for sample, mds_emb in zip(dataset_slice, mds_embedding):
#     sample[embedding_field_name] = mds_emb
#     sample.save()

In [ ]:
# # create a FO visualization with the MDS embedding
# import fiftyone.brain as fob
# results = fob.compute_visualization(
#     dataset_slice,
#     embeddings=embedding_field_name, brain_key=embedding_field_name + "_viz",
#     method="umap", num_dims=2
# )

## Keyframes with ZCore

In [ ]:
embedding_field_name = "embeddings_hausdorff_nbd_mds_8"
zcore_score_field_name = "zcore_score_hausdorff_nbd_mds_8"

### Compute ZCore scores

In [ ]:
# import fiftyone.operators as foo
# print(foo.list_errors())

In [ ]:
ctx = {
    "dataset": dataset,
    "params": {
        "embeddings": embedding_field_name,
        "zcore_score_field": zcore_score_field_name,
    },
}
foo.execute_operator("@51labs/zero-shot-coreset-selection/compute_zcore_score", ctx)

### Select

In [ ]:
SELECT_FRACTION = 0.2
SELECT_NUMBER = int(len(dataset_slice) * SELECT_FRACTION)

In [ ]:
uniform_samples = dataset_slice.sort_by("frame_number")[:SELECT_NUMBER]
random_samples = dataset_slice.take(SELECT_NUMBER)

In [ ]:
zcore_hausdorff_samples = dataset_slice.sort_by(
    zcore_score_field_name, reverse=True
)[:SELECT_NUMBER]

zcore_clip_samples = dataset_slice.sort_by(
    "zcore_score_clip", reverse=True
)[:SELECT_NUMBER]

### Assign to Exemplar

In [ ]:
def assign_to_nearest_neighbor_exemplar(
    all_samples,
    exemplar_samples,
    embedding_field,
    exemplar_indicator_field,
) -> dict:
    """
    Assign each sample to the nearest neighbor among the exemplars.
    The NN is defined by the embedding in the provided embedding_field.
    """
    if exemplar_indicator_field not in all_samples.first().field_names:
        all_samples._dataset.add_sample_field(exemplar_indicator_field, fo.DictField)

    exemplar_ids = exemplar_samples.values("id")
    exemplar_embeddings = exemplar_samples.values(embedding_field)
    for sample in all_samples:
        nnbr_index = np.argmin(
            np.linalg.norm(
                exemplar_embeddings - sample[embedding_field], axis=1
            )
        )
        nnbr_id = exemplar_ids[nnbr_index]
        # Assign the sample to the nearest neighbor
        if nnbr_id == sample.id:
            is_exemplar = True
        else:
            is_exemplar = False
        sample[exemplar_indicator_field] = {
            "is_exemplar": is_exemplar,
            "exemplar_assignment": [nnbr_id] if not is_exemplar else []
        }
    
    all_samples.save()
    return all_samples

In [ ]:
dataset_slice = assign_to_nearest_neighbor_exemplar(
    dataset_slice,
    uniform_samples,
    embedding_field_name,
    "exemplar_uniform",
)
dataset_slice = assign_to_nearest_neighbor_exemplar(
    dataset_slice,
    random_samples,
    embedding_field_name,
    "exemplar_random",
)
dataset_slice = assign_to_nearest_neighbor_exemplar(
    dataset_slice,
    zcore_hausdorff_samples,
    embedding_field_name,
    "exemplar_zcore_hausdorff",
)
dataset_slice = assign_to_nearest_neighbor_exemplar(
    dataset_slice,
    zcore_clip_samples,
    embedding_field_name,
    "exemplar_zcore_clip",
)

### Propagate + Evaluate

In [ ]:
uniform_score = propagate_annotations(
    dataset_slice,
	exemplar_frame_field="exemplar_uniform",
	input_annotation_field="ha_test_1",
	output_annotation_field="ha_test_1_propagated",
)
print(f"Score: {uniform_score}")

In [ ]:
random_score = propagate_annotations(
    dataset_slice,
	exemplar_frame_field="exemplar_random",
	input_annotation_field="ha_test_1",
	output_annotation_field="ha_test_1_propagated",
)
print(f"Score: {random_score}")

In [ ]:
zcore_hausdorff_score = propagate_annotations(
    dataset_slice,
	exemplar_frame_field="exemplar_zcore_hausdorff",
	input_annotation_field="ha_test_1",
	output_annotation_field="ha_test_1_propagated",
)
print(f"Score: {zcore_hausdorff_score}")

In [ ]:
zcore_clip_score = propagate_annotations(
    dataset_slice,
	exemplar_frame_field="exemplar_zcore_clip",
	input_annotation_field="ha_test_1",
	output_annotation_field="ha_test_1_propagated",
)
print(f"Score: {zcore_clip_score}")

## Final Workflow

1. compute `all_embeddings` with the siamese --> has spatial dimensions
2. compute the hausdorff matrix using these
3. MDS embeddings
4. Use that to find keyframes --> how? ZCore??

see `test_zcore_siamfc_e2e.py`

Next steps:

1. Sample
2. Exemplar Assignment
3. Annotation Propagation
4. Evaluation

Given a constant annotation_propagation module, and a fixed selection budget γ, compare:

- random selection
- every (1/γ)th sample
- ZCore selection with clip-derived embeddings
- ZCore selection with hausdorff-mds-derived embeddings


## Standard Datasets

In [ ]:
dataset = foz.load_zoo_dataset("coco-2017")